In [1]:
import os

os.chdir("..")

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn


In [3]:
# Import necessary modules and functions
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from src.neural_nets import create_neural_1, create_neural_3, create_neural_6, parameter_tuning_nn, runs_nn
from data.data_generator import get_data, preprocess

Regression function and data for hyperparameter tuning of the networks

In [ ]:
def regression_func(x):
    return np.exp(np.linalg.norm(x, axis=1))

input_dim = 7

x, y = get_data(regression_func, x_dim=input_dim, num_samples=100, sigma=0.05)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

train_data = preprocess(x_train, y_train, batch_size=20, training=True)
val_data = preprocess(x_val, y_val, batch_size=20, training=False)

Neural Network with one hidden Layer

In [4]:
# Hyperparameter tuning of network with 1 hidden layer

units = [5,10,25,50,75]
best_hp = parameter_tuning_nn(
    create_neural_1,
    train_data = train_data,
    test_data = val_data,
    input_dim = input_dim,
    units = units,
    epochs = 1000,
)

print(best_hp)

Unit: 5, MSE: 13.116472244262695, MAE: 3.503201723098755
Unit: 10, MSE: 0.9905808568000793, MAE: 0.8468599319458008
Unit: 25, MSE: 0.18164971470832825, MAE: 0.3196832239627838
Unit: 50, MSE: 0.19012995064258575, MAE: 0.33600401878356934
Unit: 75, MSE: 0.1763584464788437, MAE: 0.3360835313796997
{'best_hp': 75, 'mse': 0.1763584464788437}


In [5]:
unit = best_hp['best_hp']
mses_neural_1, maes_neural_1 = runs_nn(
    create_neural_1,
    input_dim=input_dim,
    regression_func=regression_func,
    units=unit,
    epochs=1000,
    batch_size=20,
    )

In [8]:
results_neural_1 = pd.DataFrame(
    {
        "MSE": mses_neural_1,
        "MAE": maes_neural_1,
    }
)

results_neural_1.to_csv("results/neural_1.csv", index=False)

Neural Network with three hidden layers

In [9]:
# Hyperparameter tuning of network with 3 hidden layer

units = [3, 6, 9, 12, 15]
best_hp = parameter_tuning_nn(
    create_neural_3,
    train_data = train_data,
    test_data = val_data,
    input_dim = input_dim,
    units = units,
    epochs=1000,
)

print(best_hp)

Unit: 3, MSE: 20.612680435180664, MAE: 4.42000675201416
Unit: 6, MSE: 0.43768349289894104, MAE: 0.5681979656219482
Unit: 9, MSE: 0.24310246109962463, MAE: 0.3963361382484436
Unit: 12, MSE: 0.3424869477748871, MAE: 0.509814977645874
Unit: 15, MSE: 0.24686332046985626, MAE: 0.4217095971107483
{'best_hp': 9, 'mse': 0.24686332046985626}


In [10]:
unit = best_hp['best_hp']
mses_neural_3, maes_neural_3 = runs_nn(
    create_neural_3,
    input_dim=input_dim,
    regression_func=regression_func,
    units=unit,
    epochs=1000,
    batch_size=20,
    )

In [11]:
results_neural_3 = pd.DataFrame(
    {
        "MSE": mses_neural_3,
        "MAE": maes_neural_3,
    }
)

results_neural_3.to_csv("results/neural_3.csv", index=False)

Neural Network with six hidden layers

In [12]:
# Hyperparameter tuning of network with 6 hidden layer

units = [2, 4, 6, 8, 10]
best_hp = parameter_tuning_nn(
    create_neural_6,
    train_data = train_data,
    test_data = val_data,
    input_dim = input_dim,
    units = units,
    epochs=1000,
)

print(best_hp)

Unit: 2, MSE: 0.5373400449752808, MAE: 0.627848744392395
Unit: 4, MSE: 0.5617508292198181, MAE: 0.6291511058807373
Unit: 6, MSE: 0.340087890625, MAE: 0.45552605390548706
Unit: 8, MSE: 0.4938618242740631, MAE: 0.5562616586685181
Unit: 10, MSE: 0.26772668957710266, MAE: 0.418938547372818
{'best_hp': 10, 'mse': 0.26772668957710266}


In [13]:
unit = best_hp['best_hp']
mses_neural_6, maes_neural_6 = runs_nn(
    create_neural_6,
    input_dim=input_dim,
    regression_func=regression_func,
    units=unit,
    epochs=1000,
    batch_size=20,
    )

In [14]:
results_neural_6 = pd.DataFrame(
    {
        "MSE": mses_neural_6,
        "MAE": maes_neural_6,
    }
)

results_neural_6.to_csv("results/neural_6.csv", index=False)